In [1]:
# COMEX Copper (High Grade) Inventory Analysis & Plotting (Plotly)
# -----------------------------------------------------------------
# Purpose:
# 1) Parse CME Copper warehouse stock Excel files
# 2) Produce trader-oriented inventory / flow / structure charts
# 3) Handle Registered (warranted) vs Eligible (non-warranted)

import os
import re
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# =========================
# 1. Path & file pattern
# =========================

DATA_DIR = r"E:\code\python\mytools\SystemMacro\CME\CME_Stocks\Copper"
FILE_PATTERN = re.compile(r"Copper_Stocks_(\d{8})\.xls")

# =========================
# 2. Read single CME Copper file
# =========================

def read_single_file(file_path, date_str):
    df = pd.read_excel(file_path, header=None)

    records = []
    current_point = None

    INVALID_KEYWORDS = ["TOTAL", "REGISTERED", "ELIGIBLE", "COPPER"]

    for _, row in df.iterrows():
        cell = str(row[0]).strip()

        # Delivery point (city)
        if (
            cell.isupper()
            and cell not in ["DELIVERY POINT"]
            and not any(k in cell for k in INVALID_KEYWORDS)
        ):
            current_point = cell
            continue

        # Category rows
        if cell.startswith("Registered") and current_point:
            category = "Registered"
        elif cell.startswith("Eligible") and current_point:
            category = "Eligible"
        elif cell == "Total" and current_point:
            category = "Total"
        else:
            continue

        try:
            ounces = float(row.dropna().iloc[-1])
        except Exception:
            continue

        records.append({
            "Date": pd.to_datetime(date_str),
            "DeliveryPoint": current_point,
            "Category": category,
            "Tons": ounces
        })

    return pd.DataFrame(records)

# =========================
# 3. Load all files
# =========================

all_dfs = []

for fname in os.listdir(DATA_DIR):
    match = FILE_PATTERN.match(fname)
    if match:
        date_str = match.group(1)
        fpath = os.path.join(DATA_DIR, fname)
        print(f"Reading {fname}")
        all_dfs.append(read_single_file(fpath, date_str))

df_all = pd.concat(all_dfs, ignore_index=True)

# =========================
# 4. Market-level aggregation
# =========================

market_df = (
    df_all
    .groupby(["Date", "Category"], as_index=False)["Tons"]
    .sum()
)

pivot_market = (
    market_df
    .pivot(index="Date", columns="Category", values="Tons")
    .sort_index()
)

pivot_market["Registered_Ratio"] = pivot_market["Registered"] / pivot_market["Total"]

pivot_market["dRegistered"] = pivot_market["Registered"].diff()
pivot_market["dEligible"] = pivot_market["Eligible"].diff()
pivot_market["dTotal"] = pivot_market["Total"].diff()

# =========================
# 5. Plot 1: Inventory levels (multi Y-axis)
# =========================

fig_inventory = go.Figure()

fig_inventory.add_trace(go.Scatter(
    x=pivot_market.index,
    y=pivot_market["Registered"],
    name="Registered (warranted)",
    yaxis="y1"
))

fig_inventory.add_trace(go.Scatter(
    x=pivot_market.index,
    y=pivot_market["Eligible"],
    name="Eligible (non-warranted)",
    yaxis="y2"
))

fig_inventory.add_trace(go.Scatter(
    x=pivot_market.index,
    y=pivot_market["Total"],
    name="Total Copper",
    yaxis="y3"
))

fig_inventory.update_layout(
    title="COMEX Copper Inventory (Multi Y-Axis, Short Tons)",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Registered"),
    yaxis2=dict(overlaying="y", side="right", title="Eligible"),
    yaxis3=dict(overlaying="y", side="right", position=0.95, title="Total"),
)

fig_inventory.show()

# =========================
# 6. Plot 2: Registered / Total ratio
# =========================

fig_ratio = px.line(
    pivot_market,
    y="Registered_Ratio",
    title="Registered / Total Ratio (Copper)"
)
fig_ratio.show()

# =========================
# 7. Plot 3: Daily net change
# =========================

fig_flow = go.Figure()
fig_flow.add_bar(x=pivot_market.index, y=pivot_market["dRegistered"], name="Δ Registered")
fig_flow.add_bar(x=pivot_market.index, y=pivot_market["dEligible"], name="Δ Eligible")
fig_flow.add_bar(x=pivot_market.index, y=pivot_market["dTotal"], name="Δ Total")

fig_flow.update_layout(
    barmode="group",
    title="Daily Inventory Net Change (Copper, tons)"
)
fig_flow.show()

# =========================
# 8. Plot 4: Registered by delivery point (stacked area)
# =========================

reg_df = df_all[df_all["Category"] == "Registered"]

reg_df = (
    reg_df
    .groupby(["Date", "DeliveryPoint"], as_index=False)["Tons"]
    .sum()
)

fig_stack = px.area(
    reg_df,
    x="Date",
    y="Tons",
    color="DeliveryPoint",
    title="Registered Copper by Delivery Point"
)
fig_stack.show()

# =========================
# 9. Plot 5: ΔRegistered heatmap (delivery point)
# =========================

reg_pivot = (
    reg_df
    .pivot(index="Date", columns="DeliveryPoint", values="Tons")
    .sort_index()
)

reg_diff = reg_pivot.diff()

fig_heat = px.imshow(
    reg_diff.T,
    aspect="auto",
    color_continuous_scale="RdBu",
    title="Δ Registered Copper by Delivery Point"
)
fig_heat.show()

# =========================
# 10. Print key inventory tables
# =========================

print("\n=== Market-level Copper Inventory (last 10 days) ===")
print(
    pivot_market[["Registered", "Eligible", "Total", "Registered_Ratio"]]
    .tail(10)
    .round(4)
)

print("\n=== Daily Net Changes (last 10 days) ===")
print(
    pivot_market[["dRegistered", "dEligible", "dTotal"]]
    .tail(10)
)

latest_date = reg_df["Date"].max()
print(f"\n=== Latest Registered by Delivery Point ({latest_date.date()}) ===")
print(
    reg_df[reg_df["Date"] == latest_date]
    .sort_values("Tons", ascending=False)
    .reset_index(drop=True)
)


Reading Copper_Stocks_20251204.xls
Reading Copper_Stocks_20251205.xls
Reading Copper_Stocks_20251208.xls
Reading Copper_Stocks_20251209.xls
Reading Copper_Stocks_20251210.xls
Reading Copper_Stocks_20251211.xls
Reading Copper_Stocks_20251212.xls
Reading Copper_Stocks_20251215.xls
Reading Copper_Stocks_20251216.xls
Reading Copper_Stocks_20251217.xls
Reading Copper_Stocks_20251218.xls
Reading Copper_Stocks_20251219.xls



=== Market-level Copper Inventory (last 10 days) ===
Category    Registered  Eligible     Total  Registered_Ratio
Date                                                        
2025-12-08    253473.0  186037.0  439510.0            0.5767
2025-12-09    255913.0  187134.0  443047.0            0.5776
2025-12-10    256200.0  188966.0  445166.0            0.5755
2025-12-11    256626.0  190672.0  447298.0            0.5737
2025-12-12    263174.0  187444.0  450618.0            0.5840
2025-12-15    263975.0  188842.0  452817.0            0.5830
2025-12-16    264902.0  189736.0  454638.0            0.5827
2025-12-17    265613.0  191373.0  456986.0            0.5812
2025-12-18    264839.0  194725.0  459564.0            0.5763
2025-12-19    265363.0  196841.0  462204.0            0.5741

=== Daily Net Changes (last 10 days) ===
Category    dRegistered  dEligible  dTotal
Date                                      
2025-12-08        200.0     2457.0  2657.0
2025-12-09       2440.0     1097.0  3537.0
